In [69]:
IMAGES_DIR = '/d2/caches/kaggle-planet/processed-images-64x64'
IMAGE_SHAPE = (64, 64, 4)

TEST_TIFFS_DIR = '/datasets/kaggle/planet/test-tif-v2'

MODEL = '/d2/caches/kaggle-planet/models/A/e09-l=0.22781-vl=0.28093.h5'

In [65]:
%run 'Lib.ipynb'

In [66]:
import keras

In [67]:
test_mapping = pd.read_csv('/datasets/kaggle/planet/test_v2_file_mapping.csv', index_col='old')

In [68]:
test_files = [x for x in list_files(TEST_TIFFS_DIR)]

In [70]:
model = keras.models.load_model(MODEL)